# **Finite sample properties of FGLS**

Let us estimate using the Feasible Generalised Least Squares method the following model:

y_i=β_0+β_1 x_1+u_i with i=1,2,...,5N

Where:

  β_0=-3

  β_1=0.8

  u_j ~ N(0,Ω⨂I_NxN)

  Ω is a 5x5 diagonal matrix whose main diagonal elements are 4, 9, 16, 25 and 36.

  x_j ~ U(1,50)

  To estimate the model, 5000 samples have been generated for the number of observations 5N equal to 5, 10, 30, 200 and 500.

The application of the FGLS method involves a number of steps:
1.   Estimate the model for each sample by Classical Least Squares.
2.   Extract its residuals and transform them
3.   Estimate a CQLS model using the residuals extracted in step 2 as the dependent variable.
4.   Extract the predicted values and transform them to use them as weights.
5.   Estimate by Weighted Least Squares the model estimated in step 1, using the weights transformed in step 4 as weights.

## Packages

In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import t, norm
from numpy.linalg import inv, cholesky

## Parameters

In [3]:
# Seed
np.random.seed(4678)

# Parameters
N = [1, 2, 6, 20, 40, 100]
num_simulations = 5000
results_df = pd.DataFrame()

## Lists for storing the results

In [4]:
# Estimated coefficients FGLS
beta_0_estimates = {j: [] for j in N}
beta_1_estimates = {j: [] for j in N}

# Estimated coefficients MCG Cholesky
beta_0_estimates_chol = {j: [] for j in N}
beta_1_estimates_chol = {j: [] for j in N}

# FGLS standard deviations
beta_0_std = {j: [] for j in N}
beta_1_std = {j: [] for j in N}

# MCG Cholesky standard deviations
beta_0_std_chol = {j: [] for j in N}
beta_1_std_chol = {j: [] for j in N}

# Test Size and Power Size
test_size_1 = {j: [] for j in N}
test_size_5 = {j: [] for j in N}
test_power_0_1 = {j: [] for j in N}
test_power_0_5 = {j: [] for j in N}
test_power_04_1 = {j: [] for j in N}
test_power_04_5 = {j: [] for j in N}

## Simulation

In [5]:
for n in N:
    Omega = np.diag(np.tile([4, 9, 16, 25, 36], n))

    # Cholesky Decomposition
    P = cholesky(inv(Omega))

    results = []

    rechazos_tf1_95 = 0
    rechazos_tf1_99 = 0

    rechazos_tf2_95 = 0
    rechazos_tf2_99 = 0

    rechazos_tf3_95 = 0
    rechazos_tf3_99 = 0

    for _ in range(num_simulations):
        # Generate values for X and U
        X = np.column_stack((np.ones(n*5), np.random.uniform(1, 50, n*5)))
        U = np.random.normal(0, np.tile([2, 3, 4, 5, 6], n), n*5)

        # Generate our Y model
        Y = X @ np.array([-3, .8]) + U

        # OLS Estimation
        model = sm.OLS(Y, X).fit()

        # Extract the residuals and square their logarithm
        model_residuals = model.resid
        logRes2 = np.log(model_residuals**2)

        """
        Estimate by OLS a model that uses as dependent variable the residuals
        that we store in logRes2. Then extract the predicted values that will
        be used to calculate the weights.
        """
        logRes2_Hat = sm.OLS(logRes2, X).fit().fittedvalues

        # Transforming weights
        weight = 1 / np.sqrt(np.exp(logRes2_Hat))

        # Estimate by Weighted Least Squares the model, using "weight" as the weight.
        wls_model = sm.WLS(Y, X, weights=weight).fit()

        # Extract the standard deviation that will allow us to calculate the t-statistics.
        std_FGLS = wls_model.bse[1]

        # Calculate the t-statistics to test the hypothesis of B_0.
        Tstat_B1_wls = abs(wls_model.params[1] - 0.8) / std_FGLS
        Tstat_B1_wls_1 = abs(wls_model.params[1]) / std_FGLS
        Tstat_B1_wls_2 = abs(wls_model.params[1] - 0.4) / std_FGLS

        """
        We found the matrix P at the beginning by using the de Cholesky decomposition.
        We now use P to calculate the variables X_chol and Y_chol.
        """
        # Pre-multiply X and Y by P
        X_chol = P @ X
        Y_chol = P @ Y

        # Estimate by OLS a model whose dependent variable is Y_chol and explanatory variable X_chol.
        ols_model_chol = sm.OLS(Y_chol, X_chol).fit()

        # Extract the deviation from the model
        std_chol = ols_model_chol.bse[1]

        # Store estimated coefficients FGLS
        beta_0_estimates[n].append(wls_model.params[0])
        beta_1_estimates[n].append(wls_model.params[1])

        # Store FGLS deviations
        beta_0_std[n].append(wls_model.bse[0])
        beta_1_std[n].append(wls_model.bse[1])

        # Store estimated coefficients MCG Cholesky
        beta_0_estimates_chol[n].append(ols_model_chol.params[0])
        beta_1_estimates_chol[n].append(ols_model_chol.params[1])

        # Store MCG Cholesky deviations
        beta_0_std_chol[n].append(ols_model_chol.bse[0])
        beta_1_std_chol[n].append(ols_model_chol.bse[1])

        if Tstat_B1_wls > t.ppf(0.995, n*5-2):
          rechazos_tf1_99 += 1
        if Tstat_B1_wls > t.ppf(0.975, n*5-2):
          rechazos_tf1_95 +=1


        if Tstat_B1_wls_1 > t.ppf(0.995, n*5-2):
          rechazos_tf2_99 += 1
        if Tstat_B1_wls_1 > t.ppf(0.975, n*5-2):
          rechazos_tf2_95 +=1


        if Tstat_B1_wls_2 > t.ppf(0.995, n*5-2):
          rechazos_tf3_99 += 1
        if Tstat_B1_wls_2 > t.ppf(0.975, n*5-2):
          rechazos_tf3_95 +=1


    # Store Test Size and Test Power
    test_size_1[n].append(rechazos_tf1_99/5000)
    test_size_5[n].append(rechazos_tf1_95/5000)
    test_power_0_1[n].append(rechazos_tf2_99/5000)
    test_power_0_5[n].append(rechazos_tf2_95/5000)
    test_power_04_1[n].append(rechazos_tf3_99/5000)
    test_power_04_5[n].append(rechazos_tf3_95/5000)

## Test Size and Test Power

In [6]:
# Create a dataframe with the results of the test sizes and power
test_results_df = pd.DataFrame(index=N, columns=['Test_Size_1', 'Test_Size_5', 'Power_Beta_0_1', 'Power_Beta_0_5', 'Power_Beta_0.4_1', 'Power_Beta_0.4_5'])

for j in N:
    test_results_df.loc[j, 'Test_Size_1'] = np.mean(test_size_1[j])
    test_results_df.loc[j, 'Test_Size_5'] = np.mean(test_size_5[j])
    test_results_df.loc[j, 'Power_Beta_0_1'] = np.mean(test_power_0_1[j])
    test_results_df.loc[j, 'Power_Beta_0_5'] = np.mean(test_power_0_5[j])
    test_results_df.loc[j, 'Power_Beta_0.4_1'] = np.mean(test_power_04_1[j])
    test_results_df.loc[j, 'Power_Beta_0.4_5'] = np.mean(test_power_04_5[j])

print(test_results_df)
excel_filename = 'sizes_power.xlsx'
test_results_df.to_excel(excel_filename)

    Test_Size_1 Test_Size_5 Power_Beta_0_1 Power_Beta_0_5 Power_Beta_0.4_1  \
1        0.0184      0.0788          0.597         0.8876           0.2086   
2         0.014      0.0634         0.9916         0.9988           0.7254   
6        0.0126      0.0532            1.0            1.0           0.9992   
20       0.0114      0.0496            1.0            1.0              1.0   
40       0.0112      0.0554            1.0            1.0              1.0   
100      0.0078      0.0454            1.0            1.0              1.0   

    Power_Beta_0.4_5  
1             0.5406  
2             0.9074  
6                1.0  
20               1.0  
40               1.0  
100              1.0  


## Statistical measures of estimated Beta FGLS

In [7]:
# Create a MultiIndex for the Beta FGLS coefficient columns.
# This will allow grouping "Mean", "Median" and "Deviation" under Beta_0 and Beta_1.
beta_columns = pd.MultiIndex.from_product([['Beta_0', 'Beta_1'], ['Media', 'Mediana', 'Desvio']],
                                           names=['', 'N'])

# Create a DataFrame to store the results of the Beta FGLS coefficients.
beta_results_df = pd.DataFrame(index=N, columns=beta_columns)

for j in N:
    beta_results_df.loc[j, ('Beta_0', 'Media')] = np.mean(beta_0_estimates[j])
    beta_results_df.loc[j, ('Beta_0', 'Mediana')] = np.median(beta_0_estimates[j])
    beta_results_df.loc[j, ('Beta_0', 'Desvio')] = np.mean(beta_0_std[j])

    beta_results_df.loc[j, ('Beta_1', 'Media')] = np.mean(beta_1_estimates[j])
    beta_results_df.loc[j, ('Beta_1', 'Mediana')] = np.median(beta_1_estimates[j])
    beta_results_df.loc[j, ('Beta_1', 'Desvio')] = np.mean(beta_1_std[j])

print(beta_results_df)
excel_filename = 'fgls.xlsx'
beta_results_df.to_excel(excel_filename)

       Beta_0                        Beta_1                    
N       Media   Mediana    Desvio     Media   Mediana    Desvio
1   -3.050148 -3.014695  3.883427  0.804025  0.802498  0.137724
2    -3.05791 -3.038393  2.744697  0.801194   0.80043  0.095921
6   -3.019007 -3.027394  1.593782   0.80047  0.801474  0.055177
20  -2.992133 -2.982704  0.874118  0.799554  0.799518  0.030043
40  -3.003301 -3.005471  0.618801  0.800199  0.800226  0.021225
100 -3.004554 -3.006429  0.391135   0.80011  0.800009  0.013415


## Statistical measures of estimated Beta MCG Cholesky

In [8]:
# Create a DataFrame to store the results of the Beta MCG coefficients.
beta_chol_results_df = pd.DataFrame(index=N, columns=beta_columns)

for j in N:
    beta_chol_results_df.loc[j, ('Beta_0', 'Media')] = np.mean(beta_0_estimates_chol[j])
    beta_chol_results_df.loc[j, ('Beta_0', 'Mediana')] = np.median(beta_0_estimates_chol[j])
    beta_chol_results_df.loc[j, ('Beta_0', 'Desvio')] = np.mean(beta_0_std_chol[j])

    beta_chol_results_df.loc[j, ('Beta_1', 'Media')] = np.mean(beta_1_estimates_chol[j])
    beta_chol_results_df.loc[j, ('Beta_1', 'Mediana')] = np.median(beta_1_estimates_chol[j])
    beta_chol_results_df.loc[j, ('Beta_1', 'Desvio')] = np.mean(beta_1_std_chol[j])

print(beta_chol_results_df)
excel_filename = 'cholesky_mcg.xlsx'
beta_chol_results_df.to_excel(excel_filename)

       Beta_0                        Beta_1                    
N       Media   Mediana    Desvio     Media   Mediana    Desvio
1   -3.020886 -3.044395  3.740923  0.803312  0.801303  0.134831
2   -3.049817 -3.008423   2.30975  0.800829  0.800353  0.081213
6   -3.027228 -3.034279  1.238463  0.800669  0.800605  0.042856
20  -3.004907 -2.983271  0.663452  0.800011  0.799637  0.022827
40  -3.005344 -3.007682   0.46692  0.800258   0.80027  0.016031
100 -3.006917  -3.00588  0.294519  0.800176  0.800048  0.010105
